# import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import sklearn

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# import dataset

In [2]:
path='/kaggle/input/coffee-prices-historical-data/coffee-prices-historical-data.csv'
coffee_prices=pd.read_csv(path)
coffee_prices.head()

,date,value
0,1973-08-20,0.6735
1,1973-08-21,0.6710
2,1973-08-22,0.6580
3,1973-08-23,0.6675
4,1973-08-24,0.6660


# data cleaning and preparing

## check data type

In [3]:
coffee_prices.dtypes

date       object
 value    float64
dtype: object

In [4]:
coffee_prices['date']=pd.to_datetime(coffee_prices['date'],infer_datetime_format=True,dayfirst=True)
coffee_prices['date']

0       1973-08-20
1       1973-08-21
2       1973-08-22
3       1973-08-23
4       1973-08-24
           ...    
12558   2023-08-21
12559   2023-08-22
12560   2023-08-23
12561   2023-08-24
12562   2023-08-25
Name: date, Length: 12563, dtype: datetime64[ns]

## check white spaces

In [5]:
coffee_prices.columns=coffee_prices.columns.str.strip()
coffee_prices.columns

Index(['date', 'value'], dtype='object')

## check null values

In [6]:
coffee_prices.isnull().sum()

date      0
value    21
dtype: int64

In [7]:
coffee_prices['value'].fillna(coffee_prices['value'].mean(),inplace=True)

In [8]:
coffee_prices.isnull().sum()

date     0
value    0
dtype: int64

## check duplicated values

In [9]:
coffee_prices.duplicated().sum()

0

## check outliers

In [10]:
coffee_prices.describe()

,value
count,12563.000000
mean,1.289773
std,0.489829
min,0.425000
25%,0.961000
50%,1.249500
75%,1.509500
max,3.356300


In [11]:
px.box(data_frame=coffee_prices,x=coffee_prices['value'],orientation='h')

## check data validity

In [12]:
coffee_prices.select_dtypes(include='object').nunique()

Series([], dtype: float64)

In [13]:
coffee_prices['date'].value_counts()

1973-08-20    1
2007-02-16    1
2007-02-21    1
2007-02-22    1
2007-02-23    1
             ..
1990-05-23    1
1990-05-24    1
1990-05-25    1
1990-05-29    1
2023-08-25    1
Name: date, Length: 12563, dtype: int64

In [14]:
coffee_prices['value'].value_counts()

1.289773    21
1.272500    15
1.263500    14
1.243500    14
1.219000    14
            ..
0.505000     1
3.186900     1
0.523500     1
3.101900     1
0.730200     1
Name: value, Length: 4829, dtype: int64

# data analysis

In [15]:
coffee_prices.head(2)

,date,value
0,1973-08-20,0.6735
1,1973-08-21,0.6710


## How much price for cup regard

In [16]:
coffee_prices.groupby('date').agg('sum')['value']

date
1973-08-20    0.673500
1973-08-21    0.671000
1973-08-22    0.658000
1973-08-23    0.667500
1973-08-24    0.666000
                ...   
2023-08-21    1.289773
2023-08-22    1.289773
2023-08-23    1.289773
2023-08-24    1.289773
2023-08-25    1.289773
Name: value, Length: 12563, dtype: float64

In [17]:
px.line(data_frame=coffee_prices,x='date',y='value')

**extract year,month,day and day name to get more insights**

In [18]:
coffee_prices['year']=coffee_prices['date'].dt.year
coffee_prices['year']

0        1973
1        1973
2        1973
3        1973
4        1973
         ... 
12558    2023
12559    2023
12560    2023
12561    2023
12562    2023
Name: year, Length: 12563, dtype: int64

In [19]:
coffee_prices['month']=coffee_prices['date'].dt.month
coffee_prices['month']

0        8
1        8
2        8
3        8
4        8
        ..
12558    8
12559    8
12560    8
12561    8
12562    8
Name: month, Length: 12563, dtype: int64

In [20]:
coffee_prices['day']=coffee_prices['date'].dt.day
coffee_prices['day']

0        20
1        21
2        22
3        23
4        24
         ..
12558    21
12559    22
12560    23
12561    24
12562    25
Name: day, Length: 12563, dtype: int64

In [21]:
coffee_prices['day_name']=coffee_prices['date'].dt.day_name()
coffee_prices['day_name']

0           Monday
1          Tuesday
2        Wednesday
3         Thursday
4           Friday
           ...    
12558       Monday
12559      Tuesday
12560    Wednesday
12561     Thursday
12562       Friday
Name: day_name, Length: 12563, dtype: object

In [22]:
coffee_prices.head()

,date,value,year,month,day,day_name
0,1973-08-20,0.6735,1973,8,20,Monday
1,1973-08-21,0.6710,1973,8,21,Tuesday
2,1973-08-22,0.6580,1973,8,22,Wednesday
3,1973-08-23,0.6675,1973,8,23,Thursday
4,1973-08-24,0.6660,1973,8,24,Friday


## How much AVG price for cup regarding to year?

In [23]:
coffee_prices[['year','value']].groupby('year').agg('mean')['value'].head()

year
1973    0.657127
1974    0.670162
1975    0.656462
1976    1.424938
1977    2.316620
Name: value, dtype: float64

In [24]:
px.line(coffee_prices[['year','value']].groupby('year').agg('mean')['value'])

## How much AVG price regarding month and year?

In [25]:
coffee_prices[['year','month','value']].groupby(['year','month']).agg('mean')['value']

year  month
1973  8        0.654840
      9        0.652700
      10       0.659552
      11       0.646810
      12       0.670937
                 ...   
2023  4        1.879125
      5        1.841761
      6        1.755845
      7        1.593848
      8        1.345741
Name: value, Length: 601, dtype: float64

In [26]:
coffee_prices.pivot_table(index='year',columns='month',values='value',aggfunc='mean')

month,1,2,3,4,5,6,7,8,9,10,11,12
year,,,,,,,,,,,,
1973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.654840,0.652700,0.659552,0.646810,0.670937
1974,0.716991,0.756550,0.732176,0.714795,0.730982,0.725355,0.705859,0.633591,0.544295,0.575364,0.602961,0.594642
1975,0.560886,0.534572,0.513710,0.480641,0.520343,0.561429,0.631441,0.846624,0.814557,0.809605,0.774787,0.839729
1976,0.931620,0.993883,1.001557,1.224490,1.356565,1.489582,1.363262,1.539273,1.570862,1.736100,1.825237,2.063243
1977,2.189590,2.425653,3.002943,3.175705,2.867505,2.626636,2.233405,2.011530,1.776133,1.612575,1.969811,1.833690
1978,1.953840,1.883276,1.586100,1.716665,1.607541,1.661732,1.263880,1.392330,1.492415,1.528181,1.456715,1.281060
1979,1.300741,1.261168,1.338636,1.425245,1.549500,1.954990,2.076700,2.052483,2.117158,2.137339,2.150419,1.925079
1980,1.722636,1.696674,1.893095,1.837386,1.977071,1.849910,1.622068,1.398719,1.329643,1.285891,1.139122,1.223810
1981,1.277950,1.209111,1.239159,1.259738,1.213100,0.971668,1.081659,1.137910,1.164571,1.349932,1.440225,1.363357


In [27]:
px.bar(coffee_prices.pivot_table(index='year',columns='month',values='value',aggfunc='mean'))

## How much (min, max, mean and quartiles Q1, Q2, Q3 ) of price for cup regarding the same month of all years?

In [28]:
coffee_prices.pivot_table(index='year',columns='month',values='value',aggfunc='mean').describe()

month,1,2,3,4,5,6,7,8,9,10,11,12
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,51.000000,50.000000,50.000000,50.000000,50.000000
mean,1.290469,1.306509,1.314896,1.319220,1.331582,1.294068,1.269873,1.258066,1.270649,1.256429,1.260795,1.275595
std,0.476192,0.500733,0.528860,0.551382,0.524849,0.492317,0.471650,0.471562,0.490032,0.460594,0.454644,0.453622
min,0.478071,0.453042,0.513710,0.480641,0.507568,0.483630,0.496833,0.482800,0.501800,0.447457,0.460275,0.468412
25%,1.019813,1.009083,1.013493,0.978441,1.045393,0.968992,0.961479,0.932280,0.908047,0.929626,0.889816,1.001577
50%,1.258437,1.241326,1.274330,1.246909,1.290789,1.251259,1.236906,1.216867,1.231196,1.237124,1.198730,1.270035
75%,1.480478,1.538283,1.458383,1.436025,1.501694,1.502785,1.559036,1.465893,1.515264,1.549081,1.545444,1.446045
max,2.410841,2.589279,3.002943,3.175705,2.867505,2.626636,2.525200,2.563709,2.590714,2.327952,2.309376,2.343870


In [29]:
describe_df=coffee_prices.pivot_table(index='year',columns='month',values='value',aggfunc='mean').describe()
describe_df

month,1,2,3,4,5,6,7,8,9,10,11,12
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,51.000000,50.000000,50.000000,50.000000,50.000000
mean,1.290469,1.306509,1.314896,1.319220,1.331582,1.294068,1.269873,1.258066,1.270649,1.256429,1.260795,1.275595
std,0.476192,0.500733,0.528860,0.551382,0.524849,0.492317,0.471650,0.471562,0.490032,0.460594,0.454644,0.453622
min,0.478071,0.453042,0.513710,0.480641,0.507568,0.483630,0.496833,0.482800,0.501800,0.447457,0.460275,0.468412
25%,1.019813,1.009083,1.013493,0.978441,1.045393,0.968992,0.961479,0.932280,0.908047,0.929626,0.889816,1.001577
50%,1.258437,1.241326,1.274330,1.246909,1.290789,1.251259,1.236906,1.216867,1.231196,1.237124,1.198730,1.270035
75%,1.480478,1.538283,1.458383,1.436025,1.501694,1.502785,1.559036,1.465893,1.515264,1.549081,1.545444,1.446045
max,2.410841,2.589279,3.002943,3.175705,2.867505,2.626636,2.525200,2.563709,2.590714,2.327952,2.309376,2.343870


In [30]:
describe_df.loc['mean',:]

month
1     1.290469
2     1.306509
3     1.314896
4     1.319220
5     1.331582
6     1.294068
7     1.269873
8     1.258066
9     1.270649
10    1.256429
11    1.260795
12    1.275595
Name: mean, dtype: float64

In [31]:
px.line(describe_df.loc['mean',:])

In [32]:
describe_df.loc['min',:]

month
1     0.478071
2     0.453042
3     0.513710
4     0.480641
5     0.507568
6     0.483630
7     0.496833
8     0.482800
9     0.501800
10    0.447457
11    0.460275
12    0.468412
Name: min, dtype: float64

In [33]:
px.line(describe_df.loc['min',:])

In [34]:
describe_df.loc['max',:]

month
1     2.410841
2     2.589279
3     3.002943
4     3.175705
5     2.867505
6     2.626636
7     2.525200
8     2.563709
9     2.590714
10    2.327952
11    2.309376
12    2.343870
Name: max, dtype: float64

In [35]:
px.line(describe_df.loc['max',:])

In [36]:
describe_df.loc['50%',:]

month
1     1.258437
2     1.241326
3     1.274330
4     1.246909
5     1.290789
6     1.251259
7     1.236906
8     1.216867
9     1.231196
10    1.237124
11    1.198730
12    1.270035
Name: 50%, dtype: float64

In [37]:
px.line(describe_df.loc['50%',:])

**the period from Feb. to Oct. is the sesonal period because it has the highest values of price**

## What is the most common day to get a cup of coffee?

In [38]:
coffee_prices['day_name'].value_counts()

Wednesday    2584
Tuesday      2568
Thursday     2534
Friday       2480
Monday       2391
Saturday        3
Sunday          3
Name: day_name, dtype: int64

In [39]:
px.bar(coffee_prices['day_name'].value_counts(),color=coffee_prices['day_name'].value_counts().index)

## How much of the most common price of a cup?

In [40]:
px.histogram(data_frame=coffee_prices,x='value')

In [41]:
# pip install ydata-profiling

In [42]:
from ydata_profiling import ProfileReport

/opt/conda/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning:

numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.

/opt/conda/lib/python3.10/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.



In [43]:
ProfileReport(coffee_prices)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

# THANK YOU